In [1]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path

In [2]:
# مسیرها
csv_path = ".\\bee\\bee_data.csv"          # مسیر فایل CSV شما
images_folder = ".\\bee\\bee_imgs"         # مسیر پوشه تصاویر

# خواندن CSV
df = pd.read_csv(csv_path)

# آماده کردن لیست ها
X = []
y = []

# پردازش هر رکورد
for idx, row in df.iterrows():
    img_name = row['file']    # نام ستون فایل عکس در CSV
    label = row['pollen_carrying']         # نام ستون تارگت در CSV

    img_path = Path(images_folder) / img_name
    img = cv2.imread(str(img_path))
    
    if img is None:
        print(f"تصویر {img_path} پیدا نشد!")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # تبدیل BGR به RGB
    img = cv2.resize(img, (128, 128))          # تغییر اندازه تصاویر به 128x128 (اختیاری)
    img = img / 255.0                          # نرمال‌سازی به [0,1]

    X.append(img)
    y.append(label)

# تبدیل به numpy array
X = np.array(X, dtype=np.float32)
y = np.array(y)

# تقسیم داده به آموزش و آزمون
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("اندازه داده‌ها:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

اندازه داده‌ها:
X_train: (4137, 128, 128, 3)
X_test: (1035, 128, 128, 3)
y_train: (4137,)
y_test: (1035,)


In [ ]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

output_activation = 'sigmoid'
loss_fn = 'binary_crossentropy'

# ساخت مدل
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation=output_activation)
])

# کامپایل مدل
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

# خلاصه مدل
model.summary()

# آموزش مدل
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    batch_size=32
)


c:\Users\qwerty\anaconda3\envs\tensorflow_env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,665 (12.61 MB)

 Trainable params: 3,305,217 (12.61 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 46s 337ms/step - accuracy: 0.9903 - loss: 0.2250 - val_accuracy: 0.9961 - val_loss: 0.2163
Epoch 2/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 43s 330ms/step - accuracy: 0.9930 - loss: 0.2778 - val_accuracy: 0.9923 - val_loss: 0.5798
Epoch 3/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 43s 328ms/step - accuracy: 0.9923 - loss: 0.3415 - val_accuracy: 0.9884 - val_loss: 0.1614
Epoch 4/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 42s 324ms/step - accuracy: 0.9932 - loss: 0.0930 - val_accuracy: 0.9961 - val_loss: 0.2260
Epoch 5/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 42s 325ms/step - accuracy: 0.9947 - loss: 0.1194 - val_accuracy: 0.9961 - val_loss: 0.5786
Epoch 6/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 43s 328ms/step - accuracy: 0.9952 - loss: 0.0475 - val_accuracy: 0.9961 - val_loss: 0.8731
Epoch 7/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 42s 325ms/step - accuracy: 0.9956 - loss: 0.1170 - val_accuracy: 0.9961 - val_loss: 0.1023
Epoch 8/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 42s 326ms/step - accuracy: 0.9959 - loss: 0

In [5]:
model.save('bee_custom.keras')